## Imports


In [16]:
import pandas as pd
import re
import numpy as np
import itertools

## Notes
Copy the extracted pose file to the data and input it here.

In [4]:
df = pd.read_csv("../1_decision_tree/data/poserac_bites.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,296.361400,293.094178,397.665245,206.603340,377.376373,432.355077,305.081375,468.451430,...,205.570562,342.267391,356.862940,358.999825,209.730660,333.227604,212.780845,341.559351,343.793752,122.978550
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,459.499241,286.945160,388.054887,193.238409,315.705841,419.375390,245.853248,464.407538,...,267.986913,348.514492,273.177822,221.686398,276.333295,348.945423,219.593627,216.220845,214.391406,266.704047
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,289.487410,285.519009,396.846670,186.351404,313.766217,371.262126,244.913861,470.215439,...,4.633939,352.081746,232.706826,234.065257,303.631607,348.217363,254.426393,347.173315,349.183821,343.030674
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,288.618273,268.979498,382.164063,173.585077,302.824455,414.413653,229.864511,459.539536,...,227.107254,352.654799,234.858961,231.754416,234.017605,347.966360,231.258714,346.965449,348.791818,231.301306
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,313.312155,300.632514,407.919343,201.106650,327.683822,401.319738,254.663514,480.387762,...,234.847301,354.146060,362.431497,362.528120,238.848604,350.278534,346.834737,348.730438,347.520903,344.869274


## Drop Padding


In [5]:
# df.drop([100], axis=0, inplace=True)
# df = df.loc[:, (df != 0).any(axis=0)]
# df.head()
# len(df)

In [6]:
# Generate new column names
# The first two columns are file_path and class
new_columns = ["file_path", "class"]
num_landmarks = (df.shape[1] - 2) // 2

for i in range(num_landmarks):
    new_columns.append(f"landmark_{i}_x")
    new_columns.append(f"landmark_{i}_y")
    # new_columns.append(f"landmark_{i}_confidence")

# Assign the new column names to the DataFrame
df.columns = new_columns

## Extract ground truth


In [7]:
def extract_salient(row):
    match = re.search(r"salient[12]", row[0])

    if match:
        salient_type = match.group()
    else:
        print("Salient type not found")

    if "salient1" == salient_type:
        return 0
    elif "salient2" == salient_type:
        return 1
    else:
        return -1


df["salient"] = df.apply(extract_salient, axis=1)
df["salient"] = df["salient"].astype(int)

df.head()

,file_path,class,landmark_0_x,landmark_0_y,landmark_1_x,landmark_1_y,landmark_2_x,landmark_2_y,landmark_3_x,landmark_3_y,...,landmark_32_y,landmark_33_x,landmark_33_y,landmark_34_x,landmark_34_y,landmark_35_x,landmark_35_y,landmark_36_x,landmark_36_y,salient
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,296.361400,293.094178,397.665245,206.603340,377.376373,432.355077,305.081375,468.451430,...,342.267391,356.862940,358.999825,209.730660,333.227604,212.780845,341.559351,343.793752,122.978550,0
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,459.499241,286.945160,388.054887,193.238409,315.705841,419.375390,245.853248,464.407538,...,348.514492,273.177822,221.686398,276.333295,348.945423,219.593627,216.220845,214.391406,266.704047,0
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,289.487410,285.519009,396.846670,186.351404,313.766217,371.262126,244.913861,470.215439,...,352.081746,232.706826,234.065257,303.631607,348.217363,254.426393,347.173315,349.183821,343.030674,0
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,288.618273,268.979498,382.164063,173.585077,302.824455,414.413653,229.864511,459.539536,...,352.654799,234.858961,231.754416,234.017605,347.966360,231.258714,346.965449,348.791818,231.301306,0
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,cow_bite,313.312155,300.632514,407.919343,201.106650,327.683822,401.319738,254.663514,480.387762,...,354.146060,362.431497,362.528120,238.848604,350.278534,346.834737,348.730438,347.520903,344.869274,0


## Extract angles between points

`angle_a_b_c` will be the angle for an angle abc.


In [24]:
# Function to calculate angle between three points (p0, p1, p2)
def calculate_angle(p0, p1, p2):
    v0 = np.array(p0) - np.array(p1)  # Vector from p1 to p0
    v2 = np.array(p2) - np.array(p1)  # Vector from p1 to p2
    cosine_angle = np.dot(v0, v2) / (np.linalg.norm(v0) * np.linalg.norm(v2))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Clip to handle precision errors
    return np.degrees(angle)

# Extract x and y coordinates from the dataframe
poses_x = df.iloc[:, 2::2].astype(float)
poses_y = df.iloc[:, 3::2].astype(float)

# Create a new dataframe to store angles
angles_df = pd.DataFrame()
angles_df['file_path'] = df['file_path']
angles_df['salient'] = df['salient']

# Calculate angles for each triplet of landmarks
num_landmarks = len(poses_x.columns) - 1

# Generate all unique triplets of landmark indices
landmark_indices = range(num_landmarks)
triplets = list(itertools.combinations(landmark_indices, 3))

# Calculate angles for all landmark triples
for count, triplet in enumerate(triplets):
    i, j, k = triplet
    p0 = (poses_x.iloc[:, i].values, poses_y.iloc[:, i].values)
    p1 = (poses_x.iloc[:, j].values, poses_y.iloc[:, j].values)
    p2 = (poses_x.iloc[:, k].values, poses_y.iloc[:, k].values)
    
    x_coords = [p0[0], p1[0], p2[0]]
    y_coords = [p0[1], p1[1], p2[1]]

    angles = [calculate_angle((x_coords[0][i], y_coords[0][i]), 
                            (x_coords[1][i], y_coords[1][i]), 
                            (x_coords[2][i], y_coords[2][i])) 
            for i in range(len(x_coords[0]))]
        
    # Create column names based on landmark indices
    col_name = f'angle_{i}_{j}_{k}'
    angles_df[col_name] = angles
    print(f"Completed {count} out of {len(triplets)}")

# The resulting dataframe will contain file_path, salient, and calculated angles
angles_df.head()

Completed 0 out of 7770
Completed 1 out of 7770
Completed 2 out of 7770
Completed 3 out of 7770
Completed 4 out of 7770
Completed 5 out of 7770
Completed 6 out of 7770
Completed 7 out of 7770
Completed 8 out of 7770
Completed 9 out of 7770
Completed 10 out of 7770
Completed 11 out of 7770
Completed 12 out of 7770
Completed 13 out of 7770
Completed 14 out of 7770
Completed 15 out of 7770
Completed 16 out of 7770
Completed 17 out of 7770
Completed 18 out of 7770
Completed 19 out of 7770
Completed 20 out of 7770
Completed 21 out of 7770
Completed 22 out of 7770
Completed 23 out of 7770
Completed 24 out of 7770
Completed 25 out of 7770
Completed 26 out of 7770
Completed 27 out of 7770
Completed 28 out of 7770
Completed 29 out of 7770
Completed 30 out of 7770
Completed 31 out of 7770
Completed 32 out of 7770
Completed 33 out of 7770
Completed 34 out of 7770
Completed 35 out of 7770
Completed 36 out of 7770
Completed 37 out of 7770
Completed 38 out of 7770
Completed 39 out of 7770
Completed 

/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe 

Completed 98 out of 7770
Completed 99 out of 7770
Completed 100 out of 7770
Completed 101 out of 7770


/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe 

Completed 102 out of 7770
Completed 103 out of 7770
Completed 104 out of 7770
Completed 105 out of 7770
Completed 106 out of 7770
Completed 107 out of 7770


...

Completed 7761 out of 7770
Completed 7762 out of 7770
Completed 7763 out of 7770
Completed 7764 out of 7770


/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe 

Completed 7765 out of 7770
Completed 7766 out of 7770
Completed 7767 out of 7770
Completed 7768 out of 7770
Completed 7769 out of 7770


/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles
/tmp/ipykernel_2675209/1548358633.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  angles_df[col_name] = angles


,file_path,salient,angle_0_1_2,angle_0_1_3,angle_0_1_4,angle_0_1_5,angle_0_1_6,angle_0_1_7,angle_0_1_8,angle_0_1_9,...,angle_32_33_34,angle_32_33_35,angle_32_33_36,angle_32_34_35,angle_32_34_36,angle_32_35_36,angle_33_34_35,angle_33_34_36,angle_33_35_36,angle_34_35_36
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,44.374562,30.037578,45.052788,73.332934,12.659892,18.180151,18.157994,30.677631,...,3.624266,0.590738,80.519527,44.819093,172.188633,126.546008,59.957391,67.412149,65.964194,51.044887
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,55.064070,64.995404,81.697327,13.045666,91.845234,56.107180,79.228773,51.135683,...,3.764130,93.480257,50.211981,63.897720,50.058365,25.976103,21.726279,35.565634,90.059490,29.030156
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,23.076981,19.114315,2.836898,65.319344,12.329033,16.014175,8.029015,46.279743,...,94.494097,73.510563,109.549008,1.956008,174.244612,178.622435,56.931031,115.357573,98.366618,3.718817
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,26.205250,16.399627,64.235546,65.737098,10.552773,25.233468,2.884013,30.332565,...,3.253777,1.878720,93.896445,54.096062,168.687471,170.658657,70.474306,44.117104,43.669326,64.481311
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,21.710153,14.792800,3.402954,60.316333,12.421984,5.349797,0.994694,16.051233,...,1.901859,37.738767,46.064435,136.795319,138.823576,102.845435,6.482030,8.510287,121.420784,100.898153


In [27]:
angles_df.to_csv("./angles.csv", index=False)

In [28]:
angles_df.head()

,file_path,salient,angle_0_1_2,angle_0_1_3,angle_0_1_4,angle_0_1_5,angle_0_1_6,angle_0_1_7,angle_0_1_8,angle_0_1_9,...,angle_32_33_34,angle_32_33_35,angle_32_33_36,angle_32_34_35,angle_32_34_36,angle_32_35_36,angle_33_34_35,angle_33_34_36,angle_33_35_36,angle_34_35_36
0,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,44.374562,30.037578,45.052788,73.332934,12.659892,18.180151,18.157994,30.677631,...,3.624266,0.590738,80.519527,44.819093,172.188633,126.546008,59.957391,67.412149,65.964194,51.044887
1,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,55.064070,64.995404,81.697327,13.045666,91.845234,56.107180,79.228773,51.135683,...,3.764130,93.480257,50.211981,63.897720,50.058365,25.976103,21.726279,35.565634,90.059490,29.030156
2,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,23.076981,19.114315,2.836898,65.319344,12.329033,16.014175,8.029015,46.279743,...,94.494097,73.510563,109.549008,1.956008,174.244612,178.622435,56.931031,115.357573,98.366618,3.718817
3,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,26.205250,16.399627,64.235546,65.737098,10.552773,25.233468,2.884013,30.332565,...,3.253777,1.878720,93.896445,54.096062,168.687471,170.658657,70.474306,44.117104,43.669326,64.481311
4,train/cow_bite/salient1/4ff8a74c52c529eceb817a...,0,21.710153,14.792800,3.402954,60.316333,12.421984,5.349797,0.994694,16.051233,...,1.901859,37.738767,46.064435,136.795319,138.823576,102.845435,6.482030,8.510287,121.420784,100.898153
